In [22]:
import numpy as np
import matplotlib.pyplot as plt
import control
import ipywidgets as widgets
from ipywidgets import interact, VBox, HBox
import sympy as sp
from IPython.display import display, Math, HTML

HTML("""
<style>
.widget-label, .custom-title {
    font-weight: bold !important;
    font-size: 16px !important;
}

.widget-toggle-buttons .widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
.widget-toggle-buttons .widget-toggle-button.selected {
    background-color: #007acc !important;
    color: white !important;
}

button.widget-toggle-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
button.widget-button {
    font-weight: bold !important;
    font-size: 15px !important;
    border-radius: 6px !important;
}
</style>
""")

In [21]:
s = sp.symbols('s')

def sympy_to_tf(expr, input_type="Step"):
    num, den = sp.fraction(sp.simplify(expr))
    num_poly = sp.Poly(num, s)
    den_poly = sp.Poly(den, s)

    if num_poly.degree() > den_poly.degree():
        q, r = sp.div(num, den, domain='QQ')
        poly_part = sp.simplify(q)
        rem_part  = sp.simplify(r/den)

        display(Math(r"\textbf{Warning: The transfer function is IMPROPER "
                     "(degree numerator > degree denominator).}"))
        display(HTML("<div style='margin:10px 0;'></div>")) 
       
        display(Math(r"\text{This means the system is non-causal and its impulse response "
                     "contains Dirac distributions } (\delta, \delta', \ldots)."))
        display(Math(r"\text{For simulation, only the strictly proper part is plotted.}"))
        display(HTML("<div style='margin:10px 0;'></div>")) 
        
        display(Math(r"\bullet \ \text{Polynomial part (non-causal, NOT plotted): } " 
                     + sp.latex(poly_part)))
        display(Math(r"\bullet \ \text{Strictly proper part (plotted): } " 
                     + sp.latex(rem_part)))

        num, den = sp.fraction(rem_part)
        num_poly = sp.Poly(num, s)
        den_poly = sp.Poly(den, s)

    if input_type == "Impulse" and num_poly.degree() == den_poly.degree():
        q, r = sp.div(num, den, domain='QQ')
        const_part = sp.simplify(q)
        prop_part  = sp.simplify(r/den)

        display(Math(r"\textbf{Warning: The transfer function has DIRECT FEEDTHROUGH (} D \neq 0 \textbf{).}"))
        display(HTML("<div style='margin:10px 0;'></div>")) 

        display(Math(r"\text{The impulse response contains a Dirac delta term that cannot be plotted.}"))
        display(HTML("<div style='margin:10px 0;'></div>")) 
            
        display(Math(r"\bullet \ \text{Feedthrough term (δ(t) contribution): } " + sp.latex(const_part)))
        display(Math(r"\bullet \ \text{Strictly proper part (plotted): } " + sp.latex(prop_part)))
        
        num, den = sp.fraction(prop_part)
        num_poly = sp.Poly(num, s)
        den_poly = sp.Poly(den, s)

    num_coeffs = [float(c) for c in num_poly.all_coeffs()]
    den_coeffs = [float(c) for c in den_poly.all_coeffs()]
    return control.TransferFunction(num_coeffs, den_coeffs)

def calculate_specs(y, t):
    y = np.asarray(y).squeeze()
    t = np.asarray(t)

    if not np.all(np.isfinite(y)) or abs(y[-1]) > 1e6:
        return None
    if len(y) >= 100 and np.std(y[-100:]) > 0.05 * (abs(y[-1]) + 1e-12):
        return None

    y0, yss = float(y[0]), float(y[-1])
    A = yss - y0
    rising = A >= 0

    y10, y90 = y0 + 0.10*A, y0 + 0.90*A
    t10 = t90 = None
    for i, yi in enumerate(y):
        if t10 is None and ((yi >= y10) if rising else (yi <= y10)):
            t10 = float(t[i])
        if t90 is None and ((yi >= y90) if rising else (yi <= y90)):
            t90 = float(t[i])
        if t10 and t90:
            break

    idx_peak = np.argmax(y) if rising else np.argmin(y)
    peak, t_peak = float(y[idx_peak]), float(t[idx_peak])

    denom = max(abs(yss), 1e-12)
    overshoot = 0.0
    if rising:
        overshoot = max(0.0, (peak - yss) / denom * 100.0)
    else:
        overshoot = max(0.0, (yss - peak) / denom * 100.0)

    # settling time (±2% band)
    tol = 0.02 * max(abs(yss), 1.0)
    last_out = None
    for i in range(len(y)):
        if abs(y[i] - yss) > tol:
            last_out = i
    if last_out is None:
        Ts = float(t[0])
    elif last_out < len(t) - 1:
        Ts = float(t[last_out + 1])
    else:
        Ts = np.nan

    return {
        "SteadyStateValue": yss,
        "t_10": t10,
        "t_90": t90,
        "Peak": peak,
        "PeakTime": t_peak,
        "Overshoot": overshoot,
        "SettlingTime": Ts
    }

def plot_response(expr_input, input_type="Step", t_end=10, show_specs=False):
    try:
        expr = sp.sympify(expr_input)
    except Exception as e:
        print("Error in the expression:", e)
        return

    try:
        system = sympy_to_tf(expr, input_type)
    except Exception as e:
        print("Error in the conversion in TransferFunction:", e)
        return

    t = np.linspace(0, t_end, 1000)

    if input_type == "Step":
        A, t0 = step_amp.value, step_time.value
        u = np.where(t >= t0, A, 0.0)
    elif input_type == "Impulse":
        A, w = imp_amp.value, imp_width.value
        u = np.where(np.abs(t - t[0]) <= w/2, A, 0.0)
    elif input_type == "Sine":
        A, f = sine_amp.value, sine_freq.value
        u = A*np.sin(2*np.pi*f*t)
    elif input_type == "Ramp":
        m, t0 = ramp_slope.value, ramp_start.value
        u = np.where(t >= t0, m*(t-t0), 0.0)
    elif input_type == "Custom":
        expr_str = custom_expr.value
        try:
            u = np.array([eval(expr_str, {"t": tt, "np": np}) for tt in t])
        except Exception as e:
            display(Math(r"\text{Error in custom input expression: }" + str(e)))
            u = np.zeros_like(t)    
    else:
        raise ValueError("Unsupported input")

    t, y = control.forced_response(system, T=t, U=u)

    plt.figure(figsize=(8, 4))
    plt.plot(t, y, label="Output y(t)", linewidth=2)
    if show_input_box.value:
        plt.plot(t, u, "--", label="Input u(t)", linewidth=1.5)

    if show_specs and input_type == "Step":
        info = calculate_specs(y, t)
        if info:
            y_ss, t10, t90, Ts = info["SteadyStateValue"], info["t_10"], info["t_90"], info["SettlingTime"]

            # Calcola livello al 10%
            y0 = float(y[0])
            A  = y_ss - y0
            y10 = y0 + 0.10 * A

            plt.axhline(y_ss, linestyle=":", color="green", label="Steady-State")
            if t10: plt.axvline(t10, linestyle="--", color="black")
            if t90: plt.axvline(t90, linestyle="--", color="black")
            if Ts and not np.isnan(Ts): plt.axvline(Ts, linestyle="-.", color="purple", label="Settling time")

            # ↔ doppia freccia Rise time
            if t10 and t90:
                plt.annotate("",
                            xy=(t10, y10), xycoords="data",
                            xytext=(t90, y10), textcoords="data",
                            arrowprops=dict(arrowstyle="<->", color="black"))
                plt.text((t10+t90)/2, y10 - 0.05, "Rise time", ha="center", va="top")


    plt.title("System Response")
    plt.xlabel("Time [s]")
    plt.ylabel("Output y(t)")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.show()

style_bold = {'description_width': 'initial'}
layout_box = widgets.Layout(width="500px", height="70px")

expr_text = widgets.Textarea(
    value="(s+1)/(s+2)", 
    description="Transfer Function:", 
    layout=layout_box, 
    style=style_bold
)
show_specs_box = widgets.ToggleButton(
    value=False, 
    description="Show Time-Domain specifications", 
    button_style='success',
    layout=widgets.Layout(width="265px", height="40px")
)    
show_input_box = widgets.ToggleButton(
    value=False,
    description="Show Input",
    button_style='success',
    layout=widgets.Layout(width="200px", height="40px")
)

input_buttons = widgets.ToggleButtons(
    options=["Step", "Impulse", "Sine", "Ramp", "Custom"],
    value="Step",
    description="Input:",
    button_style="info",
    tooltips=["Step", "Impulse", "Sine", "Ramp", "Custom"],
    style={'description_width': 'initial'}
)

step_amp   = widgets.FloatText(
    value=1.0,
    description="Step amplitude:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

step_time  = widgets.FloatText(
    value=0.0,
    description="Step start time:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

sine_amp   = widgets.FloatText(
    value=1.0,
    description="Sine amplitude:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

sine_freq  = widgets.FloatText(
    value=0.5,
    description="Sine frequency [Hz]:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

ramp_slope = widgets.FloatText(
    value=1.0,
    description="Ramp slope:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

ramp_start = widgets.FloatText(
    value=0.0,
    description="Ramp start time:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

imp_amp    = widgets.FloatText(
    value=1.0,
    description="Impulse amplitude:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

imp_width  = widgets.FloatText(
    value=0.02,
    description="Impulse width [s]:",
    layout=widgets.Layout(width="300px"),
    style={'description_width': 'initial'})

custom_expr = widgets.Textarea(
    value="np.sin(t)+0.5*t", 
    description="Custom u(t):", 
    layout=widgets.Layout(width="500px", height="70px"),
    style={'description_width': 'initial'}
)

input_params_box = VBox([])

time_label = widgets.HTML(value="<span class='custom-title'>Time:</span>")
tend_slider = widgets.FloatSlider(value=10, min=1, max=100, step=1, description="T:")

reset_button = widgets.Button(description="Reset", button_style="danger")

def reset_values(_):
    expr_text.value = "(s+1)/(s+2)"
    input_buttons.value = "Step"
    step_amp.value, step_time.value = 1.0, 0.0
    sine_amp.value, sine_freq.value = 1.0, 0.5
    ramp_slope.value, ramp_start.value = 1.0, 0.0
    imp_amp.value, imp_width.value = 1.0, 0.02
    tend_slider.value = 10
    show_specs_box.value = False
    show_input_box.value = False
    custom_expr.value = "np.sin(t)+0.5*t"

reset_button.on_click(reset_values)

def update_input_params(_=None):
    if input_buttons.value == "Step":
        input_params_box.children = [VBox([step_amp, step_time])]
    elif input_buttons.value == "Sine":
        input_params_box.children = [VBox([sine_amp, sine_freq])]
    elif input_buttons.value == "Ramp":
        input_params_box.children = [VBox([ramp_slope, ramp_start])]
    elif input_buttons.value == "Impulse":
        input_params_box.children = [VBox([imp_amp, imp_width])]
    elif input_buttons.value == "Custom":
        input_params_box.children = [VBox([custom_expr])]    

def toggle_specs_visibility(_=None):
    show_specs_box.layout.display = "" if input_buttons.value == "Step" else "none"
    if input_buttons.value != "Step": show_specs_box.value = False

input_buttons.observe(update_input_params, names="value")
input_buttons.observe(toggle_specs_visibility, names="value")
update_input_params()
toggle_specs_visibility()

ui = VBox([expr_text, input_buttons, input_params_box, time_label, tend_slider, HBox([show_specs_box, show_input_box]), reset_button])

out = widgets.Output()

def update_plot(change=None):
    with out:
        out.clear_output(wait=True)
        plot_response(
            expr_text.value,
            input_buttons.value,
            t_end=tend_slider.value,
            show_specs=show_specs_box.value
        )

for w in [
    expr_text,
    input_buttons, custom_expr,
    step_amp, step_time,
    sine_amp, sine_freq,
    ramp_slope, ramp_start,
    imp_amp, imp_width,
    show_input_box,
    show_specs_box,
    tend_slider
]:
    w.observe(update_plot, names="value")

display(ui, out)
update_plot()

Output()